<a href="https://colab.research.google.com/github/park-geun-hyeong/CUDA/blob/main/EX5_0930.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Oct  4 07:46:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


 ### blockIdx.x, threadIdx.x
- 모든 thread에 대해 주어지는 unique ID로서 설정
- Kernel 함수 호출 시 파라미터 값에 따라 자동으로 생성
- 기존 CPU 프로그래밍에 비해 스레드 관리가 쉬움

### cudaDeviceSynchronize():
- CUDA의 모든 kernels 호출은 비동기로 이루어짐
- 본 function을 사용하지 않는 경우 host는 kernel function호출 후 next instruction을 계속해서 실행
- 본 function을 사용하는 경우 host는 device가 작업을 완료할 떄까지 대기

In [12]:
%%writefile print_structure.cu

#include<stdio.h>

__global__ void print_from_gpu()
{
  printf("Hello world, [Thread Block] : [%d, %d] on device!\n",threadIdx.x, blockIdx.x);
}

int main(){

  printf("Hello world from host\n");
  print_from_gpu<<<2,2>>>();
  cudaDeviceSynchronize();
  return 0;
}

Overwriting print_structure.cu


In [13]:
!nvcc -o print_gpu print_structure.cu

In [14]:
!./print_gpu

Hello world from host
Hello world, [Thread Block] : [0, 0] on device!
Hello world, [Thread Block] : [1, 0] on device!
Hello world, [Thread Block] : [0, 1] on device!
Hello world, [Thread Block] : [1, 1] on device!


## __global__ void kernel<<<Dg, Db, Ns, S >>> ();

### Dg(dimensions of the grid: type dim3)
- 그리드의 크기
- 블록 개수를 3차원적으로 지정
- dim3형의 변수로 지정하며 생략 x
- Dg(x,y,z)로 각 차원 지정
- dim3 Dg(3,2,1)과 같이 사용
- 단일 상수로 넣을 경우 x축 값만 할당

### Db(dimensions of the block: type dim3)
- 블록의 크기
- 블록 안의 스레드 개수를 3차원적으로 지정
- dim3 형의 변수 지정하며 생략 x
- Db(x,y,z)로 지정
- dim3 Db(4,4,4)와 같이 사용
- 단일 상수로 넣을 경우 x축 값만 할당

### Ns(Number of bytes shared memory dynamically allocated/ block: type size_t)
- 각 불록에서 사용하는 shared-memory 크기지정
- 정수값으로 입력, 생략 가능
- 생략했을 때 디폴트 값으로 0 대입됨

### S(associated cudaStream_t)
- 스트림은 비동기 처리를 하기 위해 사용
- 스트림 번호를 0 이상의 정수로 지정
- 스트림 번호 0은 비동기 처리가 아닌 동기 처리를 하는 것을 의미


In [22]:
%%writefile print_structure2.cu

#include<stdio.h>

__global__ void print_from_gpu()
{
  printf("Hello world, [Block_x, Block_y, Block_z, Thread] : [%d, %d, %d, %d] on device!\n",blockIdx.x, blockIdx.y, blockIdx.z, threadIdx.x);
  // printf("gridDim: %d, blockDim: %d\n", gridDim, blockDim);
}

int main(){

  printf("Hello world from host\n");
  dim3 Dg(3,2,3);
  print_from_gpu<<<Dg,1>>>(); // 총 3x2x3의 block생성, 각 block당 1개의 Thread 생성
  cudaDeviceSynchronize();
  return 0;
}

Overwriting print_structure2.cu


In [23]:
!nvcc -o print_gpu2 print_structure2.cu

In [24]:
!./print_gpu2

Hello world from host
Hello world, [Block_x, Block_y, Block_z, Thread] : [2, 0, 2, 0] on device!
Hello world, [Block_x, Block_y, Block_z, Thread] : [0, 1, 1, 0] on device!
Hello world, [Block_x, Block_y, Block_z, Thread] : [0, 1, 2, 0] on device!
Hello world, [Block_x, Block_y, Block_z, Thread] : [2, 1, 2, 0] on device!
Hello world, [Block_x, Block_y, Block_z, Thread] : [1, 1, 0, 0] on device!
Hello world, [Block_x, Block_y, Block_z, Thread] : [1, 1, 1, 0] on device!
Hello world, [Block_x, Block_y, Block_z, Thread] : [0, 0, 2, 0] on device!
Hello world, [Block_x, Block_y, Block_z, Thread] : [1, 0, 2, 0] on device!
Hello world, [Block_x, Block_y, Block_z, Thread] : [1, 1, 2, 0] on device!
Hello world, [Block_x, Block_y, Block_z, Thread] : [2, 0, 1, 0] on device!
Hello world, [Block_x, Block_y, Block_z, Thread] : [2, 1, 0, 0] on device!
Hello world, [Block_x, Block_y, Block_z, Thread] : [2, 1, 1, 0] on device!
Hello world, [Block_x, Block_y, Block_z, Thread] : [1, 0, 1, 0] on device!
Hel